# Numerical integration with Monte Carlo

This code prices options by finding their risk-neutral expected value through the integral formula.

### (1) Non-path-dependent options

In [ ]:
import math
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm
from scipy.stats import qmc

# Code currently set to price a vanilla call

S_0 = 100
K = 100
r = 0.05
sigma = 0.2
T = 1


# With normal integration for options with 1 underlying

def Payoff(S):
    return max(S - K, 0)

def pdf(S):
    return norm.pdf((math.log(S / S_0) - (r - sigma ** 2 / 2) * T) / (sigma * math.sqrt(T))) / (S * sigma * math.sqrt(T)) # Formula derived from the Wiener process pdf using simple probability

def integrand(S):
    return Payoff(S) * pdf(S)

V = math.exp(-r * T) * quad(integrand, 0, 9999)[0] # Don't use np.inf instead of 9999, because for some reason it doesn't work
print(V)



# With Monte Carlo integration, can handle multidimensional options

def multidim_MC_integration(f, a, b, M):
    '''
    Integrates a function f from a[i] to b[i] in the i-th variable, with M points for the Monte Carlo method.

    Parameters:
        f (function): Function to integrate.
        a (tuple): Tuple with the lower integration limit of all the variables.
        b (tuple): Tuper with the upper integration limit of all the variables.
        M (int): Number of point to use for Monte Carlo.
    
    Returns:
        float: Evaluated integral
    '''
    total = 0

    # Obtain quasi-random sequence of numbers to sample function for Monte Carlo
    sampler = qmc.Halton(len(a), scramble=True)  # scramble=True for randomized Halton
    samples = sampler.random(M) # Pick first M numbers of Halton sequence
    scaled_samples = qmc.scale(samples, a, b)

    # Find average of the function by sampling the M points
    for i in range(M):
        x = tuple(scaled_samples[i])
        total = total + f(x)
    average = total / M

    # Return average times volume
    return average * np.prod(np.array(b) - np.array(a))


def Payoff_multidim(S_tuple):
    '''
    Payoff of the option given the underlyings at maturity

    Inputs:
        S_tuple (tuple): Value of underlyings
    
    Returns:
        float: Payoff of option for those S_tuple values of the underlyings at maturity
    '''
    S = S_tuple[0]
    return max(S - K, 0)

def pdf_multidim(S_tuple):
    '''
    Pdf of the underlyings at maturity

    Inputs:
        S_tuple (tuple): Value of the underlyings, inputs of the pdf
    
    Returns:
        float: Value of pdf for those function inputs
    '''
    S = S_tuple[0]
    return norm.pdf((math.log(S / S_0) - (r - sigma ** 2 / 2) * T) / (sigma * math.sqrt(T))) / (S * sigma * math.sqrt(T)) # Formula derived from the Wiener process pdf using simple probability

def integrand_multidim(S_tuple):
    '''
    Integrand function for the expected value, is the product of the pdf times the payoff

    Inputs:
        S_tuple (tuple): Value of the underlyings, inputs of the integrand function
    
    Returns:
        float: Integrand for those function inputs
    '''
    return Payoff_multidim(S_tuple) * pdf_multidim(S_tuple)

# V is the expected value of the payoff discounted to time
V = math.exp(-r * T) * multidim_MC_integration(integrand_multidim, (1e-14,), (9999,), 99999)
print(V)

10.45058357242035
10.452291386733311
